In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import predict
import glob

symbols = []
images_overall = []
datadir = 'data/MYDATA/train/'
for name in glob.glob(datadir):
    symbol = name.split('_')[1].replace(".png","")
    ioverall = io.imread(name)
    images_overall.append(ioverall)
    symbols.append(symbol)

IndexError: list index out of range